<a href="https://colab.research.google.com/github/OneFineStarstuff/Onefinebot/blob/main/Basic_hybrid_quantum_classical_variational_circuit_for_binary_classification_of_quantum_states.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane

In [ ]:
from pennylane import numpy as np
import pennylane as qml

# Define quantum device and circuit
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

def quantum_circuit(weights):
    qml.RX(weights[0], wires=0)
    qml.RY(weights[1], wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0))

# Hybrid quantum-classical model
@qml.qnode(dev)
def hybrid_model(weights):
    return quantum_circuit(weights)

# Training loop
weights = np.array([0.1, 0.2], requires_grad=True)
optimizer = qml.GradientDescentOptimizer(stepsize=0.1)

epochs = 10
for epoch in range(epochs):
    weights = optimizer.step(lambda w: (hybrid_model(w) - 1)**2, weights)
    print(f"Epoch {epoch+1}: weights = {weights}")

print("Trained weights:", weights)

In [ ]:
pip install qiskit qiskit-aer

In [ ]:
import numpy as np
import qiskit
import torch
from qiskit import Aer, QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.opflow import StateFn, PauliSumOp
from qiskit.utils import QuantumInstance
from torch.optim import Adam

# Define the quantum circuit
def create_quantum_circuit(params):
    qc = QuantumCircuit(2)
    qc.rx(params[0], 0)
    qc.rx(params[1], 1)
    qc.cx(0, 1)
    return qc

# Define the cost function
def cost_function(params, data, labels):
    qc = create_quantum_circuit(params)
    backend = Aer.get_backend('statevector_simulator')
    quantum_instance = QuantumInstance(backend)
    result = quantum_instance.execute(qc)
    statevector = result.get_statevector()
    hamiltonian = PauliSumOp.from_list([("ZI", 1), ("IZ", 1), ("ZZ", 1)])
    state_fn = StateFn(statevector, is_measurement=True)
    cost = hamiltonian @ state_fn
    predictions = torch.tensor([state_fn.coefficient for _ in data])
    loss = torch.nn.functional.cross_entropy(predictions, labels)
    return loss.item()

# Generate synthetic data
data = np.random.randn(100, 2)
labels = np.random.randint(0, 2, 100)

# Convert data to PyTorch tensors
data_tensor = torch.tensor(data, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Initialize parameters
params = [Parameter('theta0'), Parameter('theta1')]
params_tensor = torch.tensor([np.pi / 4, np.pi / 4])

# Set up optimizer
optimizer = Adam(params_tensor, lr=0.01)

# Training loop
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    loss = cost_function(params_tensor, data_tensor, labels_tensor)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

print("Trained Parameters:", params_tensor)